In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json as json
# Import API key
from config import g_key

In [2]:
#Read in weather data pulled from OpenWeather API
weather_data = pd.read_csv('../WeatherPy/output_data/cities.csv')
weather_data.head()

,City,Cloudiness,Country,Dates,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Barrow,90,US,1.583694e+09,83,71.29,-156.79,-14.8,8.05
1,Intampilan,29,PH,1.583694e+09,93,11.48,122.74,74.1,1.63
2,Jamestown,1,US,1.583694e+09,31,42.10,-79.24,55.4,8.05
3,Hobart,90,AU,1.583694e+09,76,-42.88,147.33,52.0,4.70
4,Vezirköprü,100,TR,1.583694e+09,47,41.14,35.45,55.4,2.57


In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)
#Set lat & lon as location and humidity as weight
locations = weather_data[['Lat', 'Lng']].astype(float)
humidity = weather_data['Humidity']

In [4]:
#Set gmap fig
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=humidity.max(),
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [5]:
#Created a dataframe of locations with desired weather
desired_temperatures_maxset = weather_data.loc[weather_data['Max Temp'] <= 75]
desired_temperatures = desired_temperatures_maxset.loc[weather_data['Max Temp'] > 65]
desired_cloudiness_maxset = desired_temperatures.loc[desired_temperatures['Cloudiness'] <= 50]
desired_cloudiness = desired_cloudiness_maxset.loc[desired_cloudiness_maxset['Cloudiness'] >= 5]
desired_humidity = desired_cloudiness.loc[desired_cloudiness['Humidity'] <= 50]
desired_wind = desired_humidity.loc[desired_humidity['Wind Speed'] <= 10]
desired_weather = desired_wind.drop('Dates', axis=1)
desired_weather

,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed
62,Lebu,12,CL,38,-37.62,-73.65,69.24,9.51
536,Oodweyne,10,SO,40,9.41,45.06,72.30,8.16


In [6]:
#Creating a map for hotels in desired cities
#Creating a df for hotel info
hotel_df = desired_weather.reset_index(drop=True)
hotels = []
#Parse Google Places API and return first hotel found in city to df
url= 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
cities = hotel_df['City'].values
for city in cities:
    lat = hotel_df.loc[hotel_df['City'] == city, 'Lat'].values
    lng = hotel_df.loc[hotel_df['City'] == city, 'Lng'].values
    coordinates = f'{lat[0]}, {lng[0]}'
    params = {
        'location': coordinates,
        'radius': 5000,
        'type': 'lodging',
        'language': 'en',
        'key': g_key,
    }
    response = requests.get(url, params=params).json()
    hotel_name = response['results'][0]['name']
    hotels.append(hotel_name)
hotel_df['Hotel Name'] = hotels

In [7]:
#New hotel df
hotel_df

,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Lebu,12,CL,38,-37.62,-73.65,69.24,9.51,Hostal Las Lilas
1,Oodweyne,10,SO,40,9.41,45.06,72.30,8.16,Eng Hiirey House


In [12]:
#Set hotel marker to heatmap
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
#Setting marker layer
locations = hotel_df[['Lat', 'Lng']]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
#Add layer to heatmap & Zoom to preferred location
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(heat_layer)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…